# Match statistics with player attributes
This generates the match statistics, but with player attributes and their position marked. The field is seperated in 8x22 grid. To keep things simple goalkeepers are excluded from data. Home players are even position of y, and away player are odd positions of y.

So imagining the field as a picture, then the player attributes are considered channels.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Grab dataframes for Match and Player_Attributes from Sqlite3 database.
conn = sqlite3.connect("database.sqlite")
df_match = pd.read_sql_query("select * from Match;", conn)
df_player = pd.read_sql_query("select * from Player_Attributes;", conn)
conn.close()

In [3]:
# Generate col names that have player position and api info
keep_cols = [col for col in df_match.columns.values if "_player_" in col]
keep_cols += ['stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']


# Generate cols that we want to drop (based on ones we want to keep)
drop_cols = [col for col in df_match.columns if col not in keep_cols]
# Drop goalkeeper info
drop_cols += ['home_player_1', 'away_player_1', 'home_player_X1', 'home_player_Y1', 'away_player_X1', 'away_player_Y1']

print(drop_cols)

['id', 'country_id', 'league_id', 'season', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_player_1', 'away_player_1', 'home_player_X1', 'home_player_Y1', 'away_player_X1', 'away_player_Y1']


In [4]:
# Drop columns that we generated above, do this before NaN check below
df_match.drop(drop_cols, axis=1, inplace=True)

# Cleanup dataset, drop any row with NaN values, reset indices
df_match.dropna(inplace=True)
df_match.reset_index(drop=True, inplace=True)
df_match.head()

,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X2,home_player_X3,home_player_X4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,24,2009-02-27 00:00:00,493017,8203,9987,2,1,2.0,4.0,6.0,...,38293.0,148313.0,104411.0,148314.0,37202.0,43158.0,9307.0,42153.0,32690.0,38782.0
1,24,2009-03-01 00:00:00,493019,9985,9984,4,0,2.0,4.0,6.0,...,37047.0,37021.0,38186.0,27110.0,32863.0,37957.0,37909.0,104386.0,38251.0,37065.0
2,25,2009-03-08 00:00:00,493025,9984,8342,1,3,2.0,4.0,6.0,...,21812.0,11736.0,37858.0,38366.0,37983.0,39578.0,38336.0,52280.0,27423.0,38440.0
3,25,2009-03-07 00:00:00,493027,8635,10000,2,0,2.0,4.0,6.0,...,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0
4,26,2009-03-13 00:00:00,493034,8203,8635,2,1,2.0,4.0,6.0,...,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0


In [5]:
# Drop goalkeeper stats, and stats related to set-piece (too situational to be useful)
drop_cols = [u'player_fifa_api_id', u'free_kick_accuracy', u'penalties', u'gk_diving', u'gk_handling', u'gk_kicking',
       u'gk_positioning', u'gk_reflexes']

# Drop columns that we generated above, do this before NaN check below
df_player.drop(drop_cols, axis=1, inplace=True)

# Cleanup dataset, drop any row with NaN values, reset indices
df_player.dropna(inplace=True)
df_player.reset_index(drop=True, inplace=True)
df_player.head()

,id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,...,stamina,strength,long_shots,aggression,interceptions,positioning,vision,marking,standing_tackle,sliding_tackle
0,1,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,71.0,70.0,45.0,54.0,65.0,69.0,69.0
1,2,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,71.0,70.0,45.0,54.0,65.0,69.0,69.0
2,3,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,...,54.0,76.0,35.0,63.0,41.0,45.0,54.0,65.0,66.0,69.0
3,4,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,...,54.0,76.0,34.0,62.0,40.0,44.0,53.0,62.0,63.0,66.0
4,5,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,...,54.0,76.0,34.0,62.0,40.0,44.0,53.0,62.0,63.0,66.0


## Check dataset validity
Now that we grabbed and cleaned our datasets, lets ensure that the api ids are correlated, namely all api id's used in df_match should also exist df_player. If not then drop that specific row

In [8]:
# Build the list of columns that store the api ids
player_api_cols = [col for col in df_match.columns if "_player_" in col and ("_X" not in col and "_Y" not in col )]
print(player_api_cols)

['home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']


In [12]:
for col_name in player_api_cols:
    if not np.isin(df_match[col_name].unique(), df_player['player_api_id'].unique(), assume_unique=True).all():
        print("Not a subset!!")
        np.isin(df_match[col_name].unique(), df_player['player_api_id'].unique(), assume_unique=True)
        break

Not a subset!!


In [ ]:
df_player.isna().sum()